## Introduccion



## Librerias

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


## EDA

Comenzamos leyendo el archivo "train_set.csv", primero daremos un vistazo rapido al dataset y haremos una limpieza para su posterior analisis

In [11]:
df = pd.read_csv('data/train_set.csv', low_memory=False)
print(f'Total registers: {len(df)}')
df.head()

Total registers: 700000


,trip_id,duration,start_time,end_time,start_lat,start_lon,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,start_station,end_station
0,101750280,35,2018-08-07 11:20:00,2018-08-07 11:55:00,33.748920,-118.275192,33.748920,-118.275192,06530,1.0,Round Trip,Walk-up,4127,4127
1,46560345,32,9/17/2017 17:51,9/17/2017 18:23,34.035679,-118.270813,34.047749,-118.243172,6683,0.0,One Way,Walk-up,3057,3062
2,120016336,6,2019-04-22 09:22:00,2019-04-22 09:28:00,34.046070,-118.233093,34.047749,-118.243172,06710,30.0,One Way,Monthly Pass,3022,3062
3,129547190,138,9/22/2019 11:27,9/22/2019 13:45,34.062580,-118.290092,34.059689,-118.294662,17068,1.0,One Way,One Day Pass,4304,4311
4,136619463,14,1/31/2020 17:11,1/31/2020 17:25,34.026291,-118.277687,34.021660,-118.278687,18841,30.0,One Way,Monthly Pass,4266,4443


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700000 entries, 0 to 699999
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   trip_id              700000 non-null  int64  
 1   duration             700000 non-null  int64  
 2   start_time           700000 non-null  object 
 3   end_time             700000 non-null  object 
 4   start_lat            694437 non-null  float64
 5   start_lon            694437 non-null  float64
 6   end_lat              681426 non-null  float64
 7   end_lon              681426 non-null  float64
 8   bike_id              700000 non-null  object 
 9   plan_duration        699792 non-null  float64
 10  trip_route_category  700000 non-null  object 
 11  passholder_type      697424 non-null  object 
 12  start_station        700000 non-null  int64  
 13  end_station          700000 non-null  int64  
dtypes: float64(5), int64(4), object(5)
memory usage: 74.8+ MB


De primera mano se puede observar que:
- Las columnas **start_time** y **end_time** son de tipo *str*, hay que modificar a tipo *timestamp* y con el formato indicado

In [12]:
df.describe()

,trip_id,duration,start_lat,start_lon,end_lat,end_lon,plan_duration,start_station,end_station
count,7.000000e+05,700000.000000,694437.000000,694437.000000,681426.000000,681426.000000,699792.000000,700000.000000,700000.000000
mean,1.069468e+08,37.084979,34.044952,-118.253849,34.044175,-118.259183,44.928697,3499.720464,3489.726771
std,4.497342e+07,125.302510,0.325255,2.332640,0.299721,2.129781,92.816296,615.918795,613.040769
min,8.369648e+06,1.000000,33.710979,-118.495422,33.710979,-118.495422,0.000000,3000.000000,3000.000000
25%,7.538051e+07,7.000000,34.037460,-118.280952,34.037048,-118.280952,1.000000,3031.000000,3030.000000
50%,1.179410e+08,13.000000,34.046612,-118.256980,34.046520,-118.256981,30.000000,3064.000000,3064.000000
75%,1.404178e+08,26.000000,34.051941,-118.247162,34.050911,-118.246422,30.000000,4214.000000,4214.000000
max,1.794831e+08,1440.000000,55.705528,118.238258,55.705528,37.606541,999.000000,4594.000000,4594.000000


Viendo la descripcion numerica de las columnas tambien podemos observar algunas irregularidades, 
- El total de registros son 700 000, pero ciertas columnas tienen un conteo menor a esta cantidad lo que implicar que hay campos vacios. Sin embargo nos interesa conocer la saturacion en las estaciones y esta columna contiene informacion completa, asi que por el momento conservaremos todos los datos.

Se puede observar el comportamiento de un dato atipico, en la fila *max* observamos un viaje con duracion de 1440 min tambien las coordenadas de inicio y final no coinciden con el rango de la ciudad estudiada, recortaremos este dato para que no afecte nuestro analisis.

In [25]:
df[df['duration'] == 1440]

,trip_id,duration,start_time,end_time,start_lat,start_lon,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,start_station,end_station
40,110113350,1440,2018-11-23 09:37:00,2018-11-24 16:44:00,34.014309,-118.491341,NaN,NaN,12227,1.0,One Way,Walk-up,4215,3000
231,137752372,1440,2/17/2020 13:30,2/18/2020 16:17,33.984928,-118.469963,33.984341,-118.471550,15346,1.0,One Way,One Day Pass,4347,4346
962,116879020,1440,2019-03-05 13:44:00,2019-03-06 17:03:00,34.005871,-118.429161,NaN,NaN,16079,30.0,One Way,Walk-up,4322,3000
1374,104612956,1440,2018-09-01 15:06:00,2018-09-03 16:24:00,34.037048,-118.254868,NaN,NaN,12356,1.0,One Way,Walk-up,3010,3000
2149,104549294,1440,2018-08-31 15:47:00,2018-09-02 14:42:00,34.044701,-118.252441,NaN,NaN,05974,1.0,One Way,Walk-up,3031,3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699213,152158637,1440,1/10/2021 15:21,1/11/2021 19:39,34.048401,-118.260948,NaN,NaN,5921,1.0,One Way,Walk-up,3035,3000
699294,146537846,1440,8/17/2020 7:57,8/20/2020 6:49,34.077351,-118.264549,NaN,NaN,6334,30.0,One Way,Monthly Pass,4510,3000
699385,136044849,1440,1/16/2020 17:46,1/21/2020 9:17,34.044170,-118.261169,NaN,NaN,6466,1.0,One Way,Walk-up,3074,3000
699876,168280913,1440,8/7/2021 14:52,8/9/2021 8:52,34.075611,-118.291862,NaN,NaN,6607,30.0,One Way,Monthly Pass,4401,3000


Buscando sobre este dato encontramos casi 3000 registros que marcan un viaje con la misma duracion, podriamos investigarse el porque obtenemos esta informacion, si es algun fallo u otro motivo, pero lo ignoraremos para este caso de estudio.

In [26]:
# Recortamos los datos con duracion de 1440 y actualizamos el dataset

df = df[df['duration'] != 1440]
print(len(df))

697069


In [28]:
# df[df['passholder_type'].isnull()]